In [1]:
%%time
import numpy as np
import pandas as pd

Wall time: 825 ms


In [2]:
cellinfo_jt=pd.read_csv("E:/xqxx/查询小区静态参数.csv",encoding='gbk',engine='python',usecols=['NAME','本地小区标识','小区名称','NB-IoT小区指示','频带','下行频点','上行带宽','下行带宽','小区标识','物理小区标识','小区激活状态','小区双工模式','上下行子帧配比','特殊子帧配比','根序列索引','UE最大允许发射功率(毫瓦分贝)','多RRU共小区指示','小区级参考信号端口数','小区发送和接收模式'])

In [3]:
cellinfo_tac=pd.read_csv("E:/xqxx/查看跟踪区域配置信息.csv",encoding='gbk',engine='python',usecols=['NAME','本地跟踪区域标识','运营商索引值','跟踪区域码','NB-IoT TA标识'])

In [4]:
cellinfo_pdsch=pd.read_csv("E:/xqxx/查询PDSCH配置信息.csv",encoding='gbk',engine='python',usecols=['NAME','本地小区标识','参考信号功率(0.1毫瓦分贝)'])

In [5]:
cellinfo_eNodeB_fun=pd.read_csv("E:/xqxx/查询eNodeB功能配置.csv",encoding='gbk',engine='python',usecols=['NAME','eNodeB标识'],dtype='str')

In [6]:
values={'NB-IoT小区指示':'否'}

In [7]:
cellinfo_jt.fillna(value=values,inplace=True)

In [8]:
cellinfo_jt=cellinfo_jt[cellinfo_jt['NB-IoT小区指示']=='否']

In [9]:
cellinfo_jt['NAME_本地小区标识']=cellinfo_jt['NAME'].map(str)+"_"+cellinfo_jt['本地小区标识'].map(str)

In [10]:
values1={'NB-IoT TA标识':'否'}

In [11]:
cellinfo_tac.fillna(value=values1,inplace=True)

In [12]:
cellinfo_tac=cellinfo_tac[cellinfo_tac['NB-IoT TA标识']=='否']

In [13]:
cellinfo_tac=cellinfo_tac[cellinfo_tac['运营商索引值']==0]

In [14]:
cellinfo_pdsch['NAME_本地小区标识']=cellinfo_pdsch['NAME'].map(str)+"_"+cellinfo_pdsch['本地小区标识'].map(str)

In [15]:
df0=pd.merge(cellinfo_jt,cellinfo_tac,how='left',left_on='NAME',right_on='NAME')

In [16]:
df1=pd.merge(df0,cellinfo_pdsch,how='left',left_on='NAME_本地小区标识',right_on='NAME_本地小区标识')

In [17]:
df2=pd.merge(df1,cellinfo_eNodeB_fun,how='left',left_on='NAME_x',right_on='NAME')

In [18]:
#删除不存在eNodeB标识的小区
df2.dropna(subset=['eNodeB标识'],inplace=True)

In [19]:
# 生成'eNodeB标识_小区标识'
df2['eNodeB标识_小区标识']=df2['eNodeB标识'].map(str)+"_"+df2['小区标识'].map(str)

In [20]:
# 将eNodeB标识转化为数值类型
df2['eNodeB标识'] = pd.to_numeric(df2['eNodeB标识'],errors='coerce')

In [21]:
df2['ECI']=df2['eNodeB标识'].map(int)*256+df2['小区标识']

In [22]:
df=df2[['eNodeB标识_小区标识','NAME_本地小区标识','小区名称','小区激活状态','本地小区标识_x','频带','下行频点','小区标识', '物理小区标识', '上下行子帧配比', '特殊子帧配比', '本地跟踪区域标识','跟踪区域码','NAME_x','eNodeB标识', 'UE最大允许发射功率(毫瓦分贝)', '参考信号功率(0.1毫瓦分贝)', '根序列索引','ECI', '小区级参考信号端口数', '小区发送和接收模式',
       '下行带宽', '小区双工模式' ,'多RRU共小区指示' ]]

In [23]:
df=df.rename(columns={'本地小区标识_x':'本地小区标识','NAME_x':'NAME'})

In [24]:
cellinfo_yc=pd.read_csv("E:/xqxx/异常数据.csv",encoding='gbk',engine='python',usecols=['NAME','MML命令','执行结果'])


In [25]:
cellinfo_yc=cellinfo_yc[cellinfo_yc['MML命令']=='命令-----LST CELL:;']

In [26]:
cellinfo_sq=pd.read_csv("E:/xqxx/查询小区扇区设备.csv",encoding='gbk',engine='python',usecols=['NAME','本地小区标识','扇区设备编号'])

In [27]:
cellinfo_sq['NAME_本地小区标识']=cellinfo_sq['NAME'].map(str)+"_"+cellinfo_sq['本地小区标识'].map(str)

In [28]:
cellinfo_sq=pd.merge(cellinfo_sq,df,how='left',on='NAME_本地小区标识')
cellinfo_sq=cellinfo_sq[['NAME_x','本地小区标识_x','NAME_本地小区标识','小区名称','扇区设备编号']]
cellinfo_sq.rename(columns={'NAME_x':'NAME','本地小区标识_x':'本地小区标识'},inplace=True)

In [29]:
cellinfo_no=pd.read_csv("E:/xqxx/没有查到相应的结果.csv",encoding='gbk',engine='python',usecols=['NAME','MML命令','备注'])

In [30]:
cellinfo_no=cellinfo_no[cellinfo_no['MML命令']=='LST CELL:']

In [31]:
cellinfo_no=pd.merge(cellinfo_no,cellinfo_eNodeB_fun,how='left',left_on='NAME',right_on='NAME')

In [33]:
cellinfo_no=cellinfo_no[['eNodeB标识','NAME','MML命令','备注']]

In [34]:
cellinfo_no['eNodeB标识'] = pd.to_numeric(cellinfo_no['eNodeB标识'],errors='coerce')

In [36]:
%%time
writer = pd.ExcelWriter("E:/xqxx/小区信息.xlsx")
# 设置不导出索引和将不同的Dateframe 写入多个sheet中
df.to_excel(writer,'小区基础信息',index=None)
cellinfo_yc.to_excel(writer,'断站信息',index=None)
cellinfo_sq.to_excel(writer,'扇区设备编号信息',index=None)
cellinfo_no.to_excel(writer,'站点下无小区',index=None)
writer.save()
print("小区信息.xlsx 已导出成功")

小区信息.xlsx 已导出成功
Wall time: 1min 18s
